In [1]:
import numpy as np
from PIL import Image
from mtcnn import MTCNN
from numpy import asarray
import cv2
from tensorflow.keras.models import load_model


detector = MTCNN()

In [2]:
pessoas = ['Desconhecido','Maju Coutinho', 'Seu Jorge', 'Will Smith']
num_classes = len(pessoas)

captura = r'F:\Bootcamp_DIO\desafio_reconhecer_rostos\videos\Maju.mp4'
video = cv2.VideoCapture(captura)


In [3]:
facenet = load_model('../modelos/facenet_keras.h5', compile=False)
modelo = load_model('../modelos/modelo_faces.keras')

In [4]:
def extrair_face(video, caixa, tamanho=(160, 160)):
    """
    Extrai a face de um quadro de vídeo com base nas coordenadas da caixa delimitadora e redimensiona para o tamanho especificado.

    Parâmetros:
    video (numpy.ndarray): O quadro de vídeo contendo a face.
    caixa (tuple): As coordenadas da caixa delimitadora no formato (x1, y1, largura, altura).
    tamanho (tuple): O tamanho desejado para a imagem da face (largura, altura).

    Retorna:
    numpy.ndarray: A imagem da face redimensionada.
    """

    # Converte o vídeo em um array de pixels
    pixels = asarray(video)
    # Extrai as coordenadas da caixa delimitadora
    x1, y1, largura, altura = caixa
    # Garante que as coordenadas sejam positivas
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + largura, y1 + altura

    # Extrai a região do rosto dos pixels
    rosto = pixels[y1:y2, x1:x2]

    # Converte a região do rosto em uma imagem PIL
    imagem_rosto = Image.fromarray(rosto)
    # Redimensiona a imagem do rosto para o tamanho especificado
    imagem_rosto = imagem_rosto.resize(tamanho)
    # Converte a imagem do rosto redimensionada de volta para um array
    rosto_array = asarray(imagem_rosto)
    return rosto_array

In [5]:
def obter_embedding(modelo, rosto_pixels):
    """
    Gera a embedding de uma imagem de rosto usando o modelo fornecido.

    Parâmetros:
    modelo (keras.engine.sequential.Sequential): O modelo de rede neural treinado para gerar embeddings de rostos.
    rosto_pixels (numpy.ndarray): A imagem do rosto em formato de array de pixels.

    Retorna:
    numpy.ndarray: A embedding gerada para a imagem do rosto.
    """

    # Normaliza os pixels do rosto
    rosto_pixels = rosto_pixels.astype('float32')
    media, desvio_padrao = rosto_pixels.mean(), rosto_pixels.std()
    rosto_pixels = (rosto_pixels - media) / desvio_padrao

    # Expande as dimensões da imagem do rosto para se adequar ao modelo
    amostra = np.expand_dims(rosto_pixels, axis=0)
    # Gera a embedding usando o modelo facenet
    embeddings = facenet.predict(amostra)
    # Prediz a classe do rosto usando o modelo fornecido
    yhat = modelo.predict(embeddings)
    return yhat[0]

In [6]:
# Inicia a captura de vídeo da webcam
while True:
    # Lê um quadro do vídeo
    _, quadro = video.read()
    # Detecta faces no quadro usando o detector MTCNN
    resultado = detector.detect_faces(quadro)
    if resultado:
        for pessoa in resultado:
            # Calcula a confiança da detecção da face
            confianca = pessoa['confidence']*100
            if confianca > 90:
                # Formata a confiança como uma string
                st1 = str(round(confianca, 2)) + '%'
                # Extrai as coordenadas da caixa delimitadora da face
                x, y, w, h = pessoa['box']
                # Extrai a face do quadro
                face = extrair_face(quadro, pessoa['box'])

                # Gera a embedding da face e prediz a classe da pessoa
                yhat = np.argmax(obter_embedding(modelo, face))
                # Desenha um retângulo ao redor da face detectada
                cv2.rectangle(quadro, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # Adiciona o nome da pessoa e a confiança acima do retângulo
                cv2.putText(quadro, (pessoas[yhat]+' '+ st1), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)

    # Exibe o quadro com as detecções
    cv2.imshow('Video', quadro)
    # Verifica se a tecla 'q' foi pressionada para sair do loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera a captura de vídeo e fecha todas as janelas abertas
video.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 44ms/step
